In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay
import pickle



    
def get_log_growth(df):
    with np.errstate(divide='ignore'):
        # Sort the dataframe by 'Scode' and 'Date' in ascending order
        df.sort_values(['股票代码', '日期'], inplace=True)

        # Create the 'Lag_close' column by shifting the 'Close' column by one period
        df['滞后一期收盘'] = df.groupby('股票代码')['收盘'].shift(1)

        # Calculate the 'log_growth' column using the formula ln('Close'/'Lag_close')
        df['收盘涨幅对数'] = np.log(df['收盘'] / df['滞后一期收盘'])
        return df


def get_volume_change(df):
    with np.errstate(divide='ignore'):
        # Sort the dataframe by 'Scode' and 'Date' in ascending order
        df.sort_values(['股票代码', '日期'], inplace=True)
        # Create the 'Lag_close' column by shifting the 'Close' column by one period
        df['滞后一期成交量'] = df.groupby('股票代码')['成交量'].shift(1)
        df['滞后一期成交量'] = df['滞后一期成交量'].replace(0, np.nan)
        # Calculate the percentage change in trade volume
        df['成交量变化百分比'] = (df['成交量'] - df['滞后一期成交量']) / df['滞后一期成交量'] * 100
        # Calculate the 'log_growth' column using the formula ln('Close'/'Lag_close')
        df['成交量涨幅对数'] = np.log(df['成交量'] / df['滞后一期成交量'])
        return df


def get_filtered_df(df, set_date, interval):
    # Convert set_date to the desired format
#     set_date = pd.to_datetime(set_date, format='%Y%m%d')
    set_date = pd.to_datetime(set_date, format='%Y%m%d')
    # Get the date that is 30 business days before set_date
    start_date = set_date - BDay(interval-1) 
    filtered_df = df[(df['日期'] >= start_date) & (df['日期'] <= set_date)]
    return filtered_df


def get_filtered_df_after(df, set_date, interval):
    # Convert set_date to the desired format
#     set_date = pd.to_datetime(set_date, format='%Y%m%d')
    set_date = pd.to_datetime(set_date)
    # Get the date that is 30 business days before set_date
    end_date = set_date + BDay(interval)
    filtered_df = df[(df['日期'] >= set_date) & (df['日期'] <= end_date)]
    return filtered_df

def get_today(df, set_date):
    # Convert set_date to the desired format
    set_date = pd.to_datetime(set_date, format='%Y%m%d')
    filtered_df = df[df['日期'] == set_date]
    return filtered_df



def get_growth(filtered_df):
    total_log_growth = filtered_df.groupby('股票代码')['收盘涨幅对数'].sum()
    total_growth = np.exp(total_log_growth) - 1
    return total_growth



def get_vol_growth(filtered_df):
    total_log_growth = filtered_df.groupby('股票代码')['成交量涨幅对数'].sum()
    total_growth = np.exp(total_log_growth) - 1
    return total_growth


def get_average_vol(filtered_df):
    average_vol = filtered_df.groupby('股票代码')['成交量'].mean()
    return average_vol

# 判断近期连续上涨
def whether_price_rise_in_last_n_days(filtered_df, lowwer, upper):
    # Group by 'Scode' and check if all differences are positive
    is_increasing = filtered_df.groupby('股票代码')['涨跌幅'].apply(lambda x: ((x >= lowwer) & (x < upper)).all())
    return is_increasing



# 判断近期成交量连续上涨
def whether_trade_vol_rise_in_last_n_days(filtered_df,extent):
    # Check if all percentage changes are within the range of 100% to 120%
    is_within_range = filtered_df.groupby('股票代码')['成交量变化百分比'].apply(lambda x: (x >= extent).all())
    return is_within_range

# 判断换手率大于n%
def whether_turnover_rate_larger_than(filtered_df,bottom):
    # Check if all percentage changes are within the range of 100% to 120%
    is_larger = filtered_df.groupby('股票代码')['换手率'].apply(lambda x: (x >= bottom).all())
    return is_larger

def get_n_days_latter(df,set_date,n_days):
    # Convert set_date to the desired format
    set_date = pd.to_datetime(set_date, format='%Y%m%d')
    # Get the date that is 30 business days before set_date
    future_day = set_date + BDay(n_days)
    filtered_df = df[df['日期'] == future_day]
    return filtered_df

    
# last_30_day_growth_less_than_20p, total rise in a period
def period_price_condition(last_period, R_l, R_h):
    last_period_growth = get_growth(last_period)
    last_period_growth_condition = last_period_growth[(last_period_growth >= R_l) & (last_period_growth < R_h)]
    last_period_growth_condition = list(last_period_growth_condition.index)
    set1 = set(last_period_growth_condition)
    return set1

# period_vol_condition
def period_vol_condition(last_period, R_l, R_h):
    last_period_growth = get_vol_growth(last_period)
    last_period_growth_condition = last_period_growth[(last_period_growth >= R_l) & (last_period_growth < R_h)]
    set1 = set(last_period_growth_condition.index)
    return set1

# price_rise_in_last_3_days, rise continiously
def recent_price_condition(new2_df,set_date,ndays,nday_lw,nday_up):
    # price rice in ndays1
    last_n_days = get_filtered_df(new2_df, set_date, ndays)
    price_rise_ndays = whether_price_rise_in_last_n_days(last_n_days,lowwer=nday_lw, upper=nday_up)
    price_rise_ndays = price_rise_ndays.loc[price_rise_ndays]
    price_rise_ndays = list(price_rise_ndays.index) # is a list
    set2 = set(price_rise_ndays)
    return set2


# volume_twice_times_of_montly_avg
def today_vol_condition(today, period, times):
    today_vol = get_average_vol(today)
    last_period_avg_vol = get_average_vol(period)
    vol_series = pd.concat([today_vol, last_period_avg_vol],axis=1).set_axis(['Series1', 'Series2'], axis=1)
    volume_times_of_period_avg = vol_series[vol_series['Series1'] > times*vol_series['Series2']]
    volume_times_of_period_avg = list(volume_times_of_period_avg.index)
    set3 = set(volume_times_of_period_avg)
    return set3

# price_rise_in_last_1_days
def today_price_condition(today,onelw,oneup):
    # price rice today
    price_rise_today = whether_price_rise_in_last_n_days(today,lowwer=onelw, upper=oneup)
    price_rise_today = price_rise_today.loc[price_rise_today]
    price_rise_today = list(price_rise_today.index) # is a list
    set4 = set(price_rise_today)
    return set4


def vol_rise_in_last_n_days(new2_df, set_date,ndays,extent):
    last_n_days = get_filtered_df(new2_df, set_date, ndays)
    vol_rise_in_last_n_days = whether_trade_vol_rise_in_last_n_days(last_n_days,extent=extent)
    vol_rise_in_last_n_days = vol_rise_in_last_n_days.loc[vol_rise_in_last_n_days]
    vol_rise_in_last_n_days = list(vol_rise_in_last_n_days.index) # is a list
    set5 = set(vol_rise_in_last_n_days)
    return set5
    

def today_turnover_condition(today, turnoverRate):
    turnover_larger_than = whether_turnover_rate_larger_than(today,turnoverRate)
    turnover_larger_than = turnover_larger_than[turnover_larger_than]
    set6 = set(turnover_larger_than.index)
    return set6


def get_n_days_latter(df,set_date,n_days):
    # Convert set_date to the desired format
    set_date = pd.to_datetime(set_date, format='%Y%m%d')
    # Get the date that is 30 business days before set_date
    future_day = set_date + BDay(n_days)
    filtered_df = df[df['日期'] == future_day]
    return filtered_df

def get_portfolio_return(df,set_date,scode_list,n_days):
    # buy at close price
    today = get_today(df, set_date)
    selected_scode = today[today['股票代码'].isin(scode_list)]
    buy_at = 100*selected_scode['收盘'].sum()
    # sell n days latter
    future_n_days = get_n_days_latter(df,set_date,n_days)
    selected_scode = future_n_days[future_n_days['股票代码'].isin(scode_list)]
    sell_at = 100*selected_scode['收盘'].sum()
    return (sell_at/buy_at-1)

In [2]:
# 合并dataframe
##############################
# 合并分时数据
##############################
import pickle
import efinance as ef
import pandas as pd
import numpy as np


today_df = ef.stock.get_realtime_quotes()
today_df = today_df.drop(['量比', '动态市盈率','总市值','流通市值','行情ID','市场类型','更新时间','昨日收盘'], axis=1)
today_df = today_df.rename(columns={'最新交易日': '日期'})
today_df = today_df.rename(columns={'今开': '开盘'})
today_df = today_df.rename(columns={'最新价': '收盘'})
today_df['日期'] = pd.to_datetime(today_df['日期'], format='%Y-%m-%d')
today_df.replace('-', float('nan'), inplace=True)
today_df['开盘'] = today_df['开盘'].astype(float)
today_df['最高'] = today_df['最高'].astype(float)
today_df['最低'] = today_df['最低'].astype(float)
today_df['收盘'] = today_df['收盘'].astype(float)
today_df['成交量'] = today_df['成交量'].astype(float)
today_df['成交额'] = today_df['成交额'].astype(float)
today_df['涨跌幅'] = today_df['涨跌幅'].astype(float)
today_df['涨跌额'] = today_df['涨跌额'].astype(float)
today_df['换手率'] = today_df['换手率'].astype(float)

with open('new_total_df.pkl', 'rb') as file:
    total_df = pickle.load(file)
    
new_total_df = pd.concat([total_df,today_df],axis=0)


def get_log_growth(df):
    with np.errstate(divide='ignore'):
        # Sort the dataframe by 'Scode' and 'Date' in ascending order
        df.sort_values(['股票代码', '日期'], inplace=True)
        df['收盘'] = pd.to_numeric(df['收盘'], errors='coerce')

        # Create the 'Lag_close' column by shifting the 'Close' column by one period
        df['滞后一期收盘'] = df.groupby('股票代码')['收盘'].shift(1)

        # Calculate the 'log_growth' column using the formula ln('Close'/'Lag_close')
        df['收盘涨幅对数'] = np.log(df['收盘'] / df['滞后一期收盘'])
        return df
    
def get_volume_change(df):
    with np.errstate(divide='ignore'):
        # Sort the dataframe by 'Scode' and 'Date' in ascending order
        df.sort_values(['股票代码', '日期'], inplace=True)
        df['成交量'] = pd.to_numeric(df['成交量'], errors='coerce')
        # Create the 'Lag_close' column by shifting the 'Close' column by one period
        df['滞后一期成交量'] = df.groupby('股票代码')['成交量'].shift(1)
        df['滞后一期成交量'] = df['滞后一期成交量'].replace(0, np.nan)
        # Calculate the percentage change in trade volume
        df['成交量变化百分比'] = (df['成交量'] - df['滞后一期成交量']) / df['滞后一期成交量'] * 100
        # Calculate the 'log_growth' column using the formula ln('Close'/'Lag_close')
        df['成交量涨幅对数'] = np.log(df['成交量'] / df['滞后一期成交量'])
        return df
    
new_total_df = get_log_growth(new_total_df)
new_total_df = get_volume_change(new_total_df)
new_total_df



,股票名称,股票代码,日期,开盘,收盘,最高,最低,成交量,成交额,涨跌幅,涨跌额,换手率,滞后一期收盘,收盘涨幅对数,滞后一期成交量,成交量变化百分比,成交量涨幅对数
7458,平安银行,000001,2022-07-04,14.44,14.43,14.47,14.24,834968.0,1.241453e+09,0.14,0.02,0.43,NaN,NaN,NaN,NaN,NaN
7459,平安银行,000001,2022-07-05,14.48,14.40,14.76,14.31,1115172.0,1.674719e+09,-0.21,-0.03,0.57,14.43,-0.002081,834968.0,33.558651,0.289371
7460,平安银行,000001,2022-07-06,14.32,14.02,14.42,13.95,1043335.0,1.520023e+09,-2.64,-0.38,0.54,14.40,-0.026743,1115172.0,-6.441787,-0.066586
7461,平安银行,000001,2022-07-07,14.03,13.91,14.14,13.87,730161.0,1.057697e+09,-0.78,-0.11,0.38,14.02,-0.007877,1043335.0,-30.016629,-0.356913
7462,平安银行,000001,2022-07-08,13.99,14.03,14.14,13.86,639698.0,9.288504e+08,0.86,0.12,0.33,13.91,0.008590,730161.0,-12.389459,-0.132269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,美心翼申,873833,2023-11-28,14.66,17.00,20.28,14.65,78901.0,1.350266e+08,7.59,1.20,29.76,15.80,0.073203,71012.0,11.109390,0.105345
5433,美心翼申,873833,2023-11-29,16.88,15.28,17.78,14.50,58721.0,9.403090e+07,-10.12,-1.72,22.15,17.00,-0.106669,78901.0,-25.576355,-0.295396
0,美心翼申,873833,2023-11-30,14.69,14.25,15.67,14.13,30554.0,4.482326e+07,-6.74,-1.03,11.53,15.28,-0.069788,58721.0,-47.967507,-0.653302
0,美心翼申,873833,2023-12-01,14.23,13.18,14.62,13.15,27368.0,3.743917e+07,-7.51,-1.07,10.32,14.25,-0.078056,30554.0,-10.427440,-0.110121


In [21]:
######################   当日策略 ######################


import time

# set_date = '20230926'
# day = pd.to_datetime(set_date, format='%Y%m%d')
    

import pandas as pd

# Assuming you have the start and end tactic dates as strings
start_tactic_date = '20231101' #str(input('输入测试开始时间：（例：20231001）'))
end_tactic_date =  str(input('输入今天的日期：（例：20231130）'))
# Convert the tactic dates to datetime objects
start_date = pd.to_datetime(start_tactic_date, format='%Y%m%d')
end_date = pd.to_datetime(end_tactic_date, format='%Y%m%d')
# Generate the series of working days
working_days = pd.bdate_range(start=start_date, end=end_date)
national_holiday_list = ['2023-09-29','2023-10-02','2023-10-03','2023-10-04','2023-10-05','2023-10-06']
May_first = ['2023-05-01','2023-05-02','2023-05-03']
Qingming = ['2023-04-05']
Spring_festival = ['2023-01-01','2023-01-02','2023-01-03','2023-01-23','2023-01-24','2023-01-25','2023-01-26','2023-01-27']
Dragon_boat = ['2023-6-22']
all_holiday = national_holiday_list+May_first+Qingming+Spring_festival+Dragon_boat

for day in all_holiday:
    if day in working_days:
        working_days = working_days.drop(day)

  
# 涨幅限制用于切片，涨幅天数是最大限制。 

start_time_original = time.time()
# portfolio args
return_rate_total = []
day_list = []
code_list = []


nday_sold = 1 #int(input("请设置在购买后第几天卖出："))
result_df = pd.DataFrame(columns=['股票代码','股票名称','日期','开盘','收盘','换手率','最高','成交量','今日涨幅','近三日涨幅','Ntimes']).set_index('股票代码')

M1 = 1000 # 过去十天涨幅不高于M1%
M2 = 1000 # 过去30天涨幅不高于M2%
N = 103 # 今日收盘价/开盘价>N%
N1 = 102 # 今日开盘价/昨日收盘价>N1%
N2 = 105 # 今日收盘价/昨日收盘价>N2%
N3 = 97 # 今日收盘价/今日最高价>N3%
X1 = 19.5 # 如果为创业板(股票代码30开头)今日涨幅<X1
X2 = 9.5 # 如果为主板(股票代码60或00开头)今日涨幅<X2
Ntimes = 1 # 今天的量大于过去10天平均量的N倍


for index, day in enumerate(working_days):
    start_time = time.time()
    slide_df = get_filtered_df(new_total_df, working_days[index], 40)
    print('Selecting stocks at {}'.format(day))
    # 计算今天的情况
    today = get_today(slide_df, day)
    today_vol = get_average_vol(today)
    today_growth = get_growth(today)
    se1, set2, set3, set4, set5, set6, set7, set8, set9 = set(), set(), set(), set(), set(), set(), set(), set(), set()
    set1_1, set1_2 = set(), set()
    
    # 过去十天涨幅不高于M1%
    slide_minus10to0 = get_filtered_df(slide_df, working_days[index-1], 10) # day - BDay(3)
    set1 = period_price_condition(slide_minus10to0, -1000,M1/100)
    
    if set1:
        # 过去30天涨幅不高于M2%
        last_30_days = get_filtered_df(slide_df, working_days[index-1], 30) #  day - BDay(prek)
        set2 = period_price_condition(last_30_days, -1000,M2/100)
        set1 = set1.intersection(set2)
    if set2:
        # 今日收盘价/开盘价>N%
        set3 = set(today[today['收盘']/today['开盘']>N/100]['股票代码'])
        set1 = set1.intersection(set3)
    if set3:
        # 今日开盘价/昨日收盘价>N1%
        set4 = set(today[today['开盘']/today['滞后一期收盘']>N1/100]['股票代码'])
        set1 = set1.intersection(set4)
    if set4:
        # 今日收盘价/昨日收盘价>N2%
        set5 = set(today[today['收盘']/today['滞后一期收盘']>N2/100]['股票代码'])
        set1 = set1.intersection(set5)
    if set5:
        # 今日收盘价/今日最高价>N3%
        set6 = set(today[today['收盘']/today['最高']>N3/100]['股票代码'])
        set1 = set1.intersection(set6)
        
    if set6:
         # 最近一段时间成交量条件：V0为D0之前10天成交量的均值的N倍以上
        set_period = get_filtered_df(slide_df, working_days[index], 10) #  day - BDay(prek)
        period_vol_avg = get_average_vol(set_period)
        vol_series = pd.concat([today_vol, period_vol_avg],axis=1).set_axis(['Series1', 'Series2'], axis=1)
        volume_n_times_of_period_avg = vol_series[vol_series['Series1'] > Ntimes*vol_series['Series2']]
        set7 = set(volume_n_times_of_period_avg.index)
        set1 = set1.intersection(set7)
    
    if set7:
        # 如果为创业板(股票代码30开头)今日涨幅（今收/昨收-1）<X1
        set8 = set(today[(today['收盘']/today['滞后一期收盘']-1) < X1/100]['股票代码'])
        set8 = {code for code in set8 if code.startswith('30')}
        set1_1 = set1.intersection(set8)
        
    if set7:
        # 如果为主板(股票代码60或00开头)今日涨幅<X2
        set9 = set(today[(today['收盘']/today['滞后一期收盘']-1) < X2/100]['股票代码'])
        set9 = {code for code in set9 if (code.startswith('60') or code.startswith('00'))}
        set1_2 = set1.intersection(set9)
        
    selected_codes = set1_1 | set1_2
        
        
    if selected_codes:
        day_list.append(day)
        code_list.append(selected_codes)
        company_list = today[['股票代码','股票名称','日期','开盘','收盘','换手率','最高','成交量']].set_index('股票代码') 
        company_list = company_list.loc[company_list.index.isin(selected_codes)]       
        today_pchange = pd.DataFrame(today_growth).rename(columns={'收盘涨幅对数': '今日涨幅'})
        today_pchange = today_pchange.loc[today_pchange.index.isin(selected_codes)]
        recent3daychange = pd.DataFrame(get_growth(get_filtered_df(slide_df, working_days[index], 3))).rename(columns={'收盘涨幅对数': '近三日涨幅'})
        recent3daychange = recent3daychange.loc[recent3daychange.index.isin(selected_codes)]
        value_of_ntimes = vol_series['Series1']/vol_series['Series2']
        value_of_ntimes = pd.DataFrame(value_of_ntimes)
        value_of_ntimes.columns = ['Ntimes']
        value_of_ntimes = value_of_ntimes.loc[value_of_ntimes.index.isin(selected_codes)]
        result_series = pd.concat([company_list,today_pchange,recent3daychange,value_of_ntimes],axis=1)
        result_df = pd.concat([result_df,result_series],axis=0)
        print('Selected codes are {}'.format(selected_codes))
    else:
        print('No stock is selected')
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time: ", execution_time, " seconds")
    print('\n')
    
end_time_all = time.time()
execution_time_all = end_time_all-start_time_original

print('Finish execution, using {} seconds'.format(execution_time_all))
# if return_rate_total:
#     average_return = sum(return_rate_total) / len(return_rate_total)
#     print('Average return: {}'.format(average_return))
#     portfolio_dict = {
#         'Date':day_list,
#         'Scode':code_list,
#         'return_rate':return_rate_total
#     }
#     portfolio_df = pd.DataFrame(portfolio_dict)
#     portfolio_df.to_excel('portfolio_return.xlsx')
result_df['今日涨幅'] = result_df['今日涨幅'].astype(float)
result_df['近三日涨幅'] = result_df['近三日涨幅'].astype(float)
result_df['Ntimes'] = result_df['Ntimes'].astype(float)
result_df['今日涨幅'] = result_df['今日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['近三日涨幅'] = result_df['近三日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['Ntimes'] = result_df['Ntimes'].map(lambda x: "{:.2f}".format(x))
result_df['日期'] = pd.to_datetime(result_df['日期'], format='%Y-%m-%d')
result_df.to_excel('select_df_tactic2.xlsx')
# else:
#     print('No stock selected')

输入今天的日期：（例：20231130）20231204
Selecting stocks at 2023-11-01 00:00:00
No stock is selected
Execution time:  0.057108402252197266  seconds


Selecting stocks at 2023-11-02 00:00:00
No stock is selected
Execution time:  0.12346434593200684  seconds


Selecting stocks at 2023-11-03 00:00:00
Selected codes are {'301328', '603567', '300398', '300607', '001314', '300580', '301261', '300660', '603667', '600580', '603666', '300280', '603416', '300258', '000837'}
Execution time:  0.14531278610229492  seconds


Selecting stocks at 2023-11-06 00:00:00
Selected codes are {'603127', '000002', '603662', '300559', '300759', '301262', '300218', '603327', '300765', '300606', '600513', '600155', '300624', '603598', '002979', '300083', '301052', '300725', '002821', '301051', '300709'}
Execution time:  0.13206815719604492  seconds


Selecting stocks at 2023-11-07 00:00:00
Selected codes are {'300364'}
Execution time:  0.1422419548034668  seconds


Selecting stocks at 2023-11-08 00:00:00
Selected codes are 

<ipython-input-21-e48261991854>:160: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  result_df.to_excel('select_df_tactic2.xlsx')


In [29]:
value_of_ntimes.columns = ['Ntimes']
value_of_ntimes

,Ntimes
股票代码,
002553,2.753197
002747,2.573537
002979,3.309485
300904,1.053929
301261,2.541896
301510,1.440627
301512,1.393540


In [23]:
######################   新的回测   ######################


import time

# set_date = '20230926'
# day = pd.to_datetime(set_date, format='%Y%m%d')
    

import pandas as pd

# Assuming you have the start and end tactic dates as strings
start_tactic_date = '20221201' #str(input('输入测试开始时间：（例：20231001）'))
end_tactic_date =  '20231127' #str(input('输入测试结束时间：（例：20231031）'))
# Convert the tactic dates to datetime objects
start_date = pd.to_datetime(start_tactic_date, format='%Y%m%d')
end_date = pd.to_datetime(end_tactic_date, format='%Y%m%d')
# Generate the series of working days
working_days = pd.bdate_range(start=start_date, end=end_date)
national_holiday_list = ['2023-09-29','2023-10-02','2023-10-03','2023-10-04','2023-10-05','2023-10-06']
May_first = ['2023-05-01','2023-05-02','2023-05-03']
Qingming = ['2023-04-05']
Spring_festival = ['2023-01-01','2023-01-02','2023-01-03','2023-01-23','2023-01-24','2023-01-25','2023-01-26','2023-01-27']
Dragon_boat = ['2023-6-22']
all_holiday = national_holiday_list+May_first+Qingming+Spring_festival+Dragon_boat

for day in all_holiday:
    if day in working_days:
        working_days = working_days.drop(day)

  
# 涨幅限制用于切片，涨幅天数是最大限制。 

start_time_original = time.time()
# portfolio args
return_rate_total = []
day_list = []
code_list = []


nday_sold = 1 #int(input("请设置在购买后第几天卖出："))
result_df = pd.DataFrame(columns=['股票代码','股票名称','日期','开盘','收盘','换手率','最高','成交量','今日涨幅','近三日涨幅','Ntimes','D+1涨幅','D+2涨幅','D+3涨幅','D+4涨幅','D+5涨幅','D+6涨幅','D+7涨幅','D+8涨幅','D+9涨幅','D+10涨幅','D+11涨幅','D+12涨幅','D+13涨幅','D+14涨幅','D+15涨幅','D+16涨幅','D+17涨幅','D+18涨幅','D+19涨幅','D+20涨幅']).set_index('股票代码')

M1 = 1000 # 过去十天涨幅不高于M1%
M2 = 1000 # 过去30天涨幅不高于M2%
N = 103 # 今日收盘价/开盘价>N%
N1 = 102 # 今日开盘价/昨日收盘价>N1%
N2 = 105 # 今日收盘价/昨日收盘价>N2%
N3 = 97 # 今日收盘价/今日最高价>N3%
X1 = 19.5 # 如果为创业板(股票代码30开头)今日涨幅<X1
X2 = 9.5 # 如果为主板(股票代码60或00开头)今日涨幅<X2
Ntimes = 1 # 今天的量大于过去10天平均量的N倍

date_string = '20231030'
timestamp = pd.to_datetime(date_string, format='%Y%m%d')
end_index = working_days.get_loc(timestamp)

for index, day in enumerate(working_days[:end_index]):
    start_time = time.time()
    slide_df = get_filtered_df(new_total_df, working_days[index], 30)
    print('Selecting stocks at {}'.format(day))
    # 计算今天的情况
    today = get_today(slide_df, day)
    today_vol = get_average_vol(today)
    today_growth = get_growth(today)
    se1, set2, set3, set4, set5, set6, set7, set8, set9 = set(), set(), set(), set(), set(), set(), set(), set(), set()
    
    # 过去十天涨幅不高于M1%
    slide_minus10to0 = get_filtered_df(slide_df, working_days[index], 10) # day - BDay(3)
    set1 = period_price_condition(slide_minus10to0, -1000,M1/100)
    
    if set1:
        # 过去30天涨幅不高于M2%
        set2 = period_price_condition(slide_df, -1000,M2/100)
        set1 = set1.intersection(set2)
    if set2:
        # 今日收盘价/开盘价>N%
        set3 = set(today[today['收盘']/today['开盘']>N/100]['股票代码'])
        set1 = set1.intersection(set3)
    if set3:
        # 今日开盘价/昨日收盘价>N1%
        set4 = set(today[today['开盘']/today['滞后一期收盘']>N1/100]['股票代码'])
        set1 = set1.intersection(set4)
    if set4:
        # 今日收盘价/昨日收盘价>N2%
        set5 = set(today[today['收盘']/today['滞后一期收盘']>N2/100]['股票代码'])
        set1 = set1.intersection(set5)
    if set5:
        # 今日收盘价/今日最高价>N3%
        set6 = set(today[today['收盘']/today['最高']>N3/100]['股票代码'])
        set1 = set1.intersection(set6)
    if set6:
         # 最近一段时间成交量条件：V0为D0之前10天成交量的均值的N倍以上
        set_period = get_filtered_df(slide_df, working_days[index], 10) #  day - BDay(prek)
        period_vol_avg = get_average_vol(set_period)
        vol_series = pd.concat([today_vol, period_vol_avg],axis=1).set_axis(['Series1', 'Series2'], axis=1)
        volume_n_times_of_period_avg = vol_series[vol_series['Series1'] > Ntimes*vol_series['Series2']]
        set7 = set(volume_n_times_of_period_avg.index)
        set1 = set1.intersection(set7)
    
    if set7:
        # 如果为创业板(股票代码30开头)今日涨幅（今收/昨收-1）<X1
        set8 = set(today[(today['收盘']/today['滞后一期收盘']-1) < X1/100]['股票代码'])
        set8 = {code for code in set8 if code.startswith('30')}
        set1_1 = set1.intersection(set8)
    if set7:
        # 如果为主板(股票代码60或00开头)今日涨幅<X2
        set9 = set(today[(today['收盘']/today['滞后一期收盘']-1) < X2/100]['股票代码'])
        set9 = {code for code in set9 if (code.startswith('60') or code.startswith('00'))}
        set1_2 = set1.intersection(set9)
        
    selected_codes = set1_1 | set1_2
        
        
    if selected_codes:
        return_df = get_filtered_df(new_total_df,working_days[index+20],25) # day+BDay(20)
        return_rate = get_portfolio_return(new_total_df,day,selected_codes,nday_sold)
        day_list.append(day)
        code_list.append(selected_codes)
        return_rate_total.append(return_rate)   
        company_list = today[['股票代码','股票名称','日期','开盘','收盘','换手率','最高','成交量']].set_index('股票代码')
        company_list = company_list.loc[company_list.index.isin(selected_codes)]       
        today_pchange = pd.DataFrame(today_growth).rename(columns={'收盘涨幅对数': '今日涨幅'})
        today_pchange = today_pchange.loc[today_pchange.index.isin(selected_codes)]
        recent3daychange = pd.DataFrame(get_growth(get_filtered_df(slide_df, working_days[index], 3))).rename(columns={'收盘涨幅对数': '近三日涨幅'})
        recent3daychange = recent3daychange.loc[recent3daychange.index.isin(selected_codes)]
        value_of_ntimes = vol_series['Series1']/vol_series['Series2']
        value_of_ntimes = pd.DataFrame(value_of_ntimes)
        value_of_ntimes.columns = ['Ntimes']
        value_of_ntimes = value_of_ntimes.loc[value_of_ntimes.index.isin(selected_codes)]
        D1_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df,working_days[index+1],1))).rename(columns={'收盘涨幅对数': 'D+1涨幅'}) # day+BDay(1)
        D1_pchange = D1_pchange.loc[D1_pchange.index.isin(selected_codes)]
        D2_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df,working_days[index+2],2))).rename(columns={'收盘涨幅对数': 'D+2涨幅'}) # day+BDay(2)
        D2_pchange = D2_pchange.loc[D2_pchange.index.isin(selected_codes)]
        D3_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df,working_days[index+3] ,3))).rename(columns={'收盘涨幅对数': 'D+3涨幅'}) # day+BDay(3)
        D3_pchange = D3_pchange.loc[D3_pchange.index.isin(selected_codes)]
        D4_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+4],4))).rename(columns={'收盘涨幅对数': 'D+4涨幅'}) # day+BDay(4)
        D4_pchange = D4_pchange.loc[D4_pchange.index.isin(selected_codes)]
        D5_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+5] ,5))).rename(columns={'收盘涨幅对数': 'D+5涨幅'}) # day+BDay(5)
        D5_pchange = D5_pchange.loc[D5_pchange.index.isin(selected_codes)]
        D6_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+6] ,6))).rename(columns={'收盘涨幅对数': 'D+6涨幅'}) # day+BDay(6)
        D6_pchange = D6_pchange.loc[D6_pchange.index.isin(selected_codes)]
        D7_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+7] ,7))).rename(columns={'收盘涨幅对数': 'D+7涨幅'}) # day+BDay(7)
        D7_pchange = D7_pchange.loc[D7_pchange.index.isin(selected_codes)]
        D8_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+8] ,8))).rename(columns={'收盘涨幅对数': 'D+8涨幅'}) # day+BDay(8)
        D8_pchange = D8_pchange.loc[D8_pchange.index.isin(selected_codes)]
        D9_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+9] ,9))).rename(columns={'收盘涨幅对数': 'D+9涨幅'}) # day+BDay(9)
        D9_pchange = D9_pchange.loc[D9_pchange.index.isin(selected_codes)]
        D10_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+10] ,10))).rename(columns={'收盘涨幅对数': 'D+10涨幅'}) # day+BDay(10)
        D10_pchange = D10_pchange.loc[D10_pchange.index.isin(selected_codes)]
        D11_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+11] ,11))).rename(columns={'收盘涨幅对数': 'D+11涨幅'}) # day+BDay(11)
        D11_pchange = D11_pchange.loc[D11_pchange.index.isin(selected_codes)]
        D12_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+12] ,12))).rename(columns={'收盘涨幅对数': 'D+12涨幅'}) # day+BDay(12)
        D12_pchange = D12_pchange.loc[D12_pchange.index.isin(selected_codes)]
        D13_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+13] ,13))).rename(columns={'收盘涨幅对数': 'D+13涨幅'}) # day+BDay(13)
        D13_pchange = D13_pchange.loc[D13_pchange.index.isin(selected_codes)]
        D14_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+14] ,14))).rename(columns={'收盘涨幅对数': 'D+14涨幅'}) # day+BDay(14)
        D14_pchange = D14_pchange.loc[D14_pchange.index.isin(selected_codes)]
        D15_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+15] ,15))).rename(columns={'收盘涨幅对数': 'D+15涨幅'}) # day+BDay(15)
        D15_pchange = D15_pchange.loc[D15_pchange.index.isin(selected_codes)]
        D16_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+16] ,16))).rename(columns={'收盘涨幅对数': 'D+16涨幅'}) # day+BDay(16)
        D16_pchange = D16_pchange.loc[D16_pchange.index.isin(selected_codes)]
        D17_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+17] ,17))).rename(columns={'收盘涨幅对数': 'D+17涨幅'}) # day+BDay(17)
        D17_pchange = D17_pchange.loc[D17_pchange.index.isin(selected_codes)]
        D18_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+18] ,18))).rename(columns={'收盘涨幅对数': 'D+18涨幅'}) # day+BDay(18)
        D18_pchange = D18_pchange.loc[D18_pchange.index.isin(selected_codes)]
        D19_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+19] ,19))).rename(columns={'收盘涨幅对数': 'D+19涨幅'}) # day+BDay(19)
        D19_pchange = D19_pchange.loc[D19_pchange.index.isin(selected_codes)]
        D20_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+20] ,20))).rename(columns={'收盘涨幅对数': 'D+20涨幅'}) # day+BDay(20)
        D20_pchange = D20_pchange.loc[D20_pchange.index.isin(selected_codes)]
        result_series = pd.concat([company_list,today_pchange,recent3daychange,value_of_ntimes, D1_pchange,D2_pchange,D3_pchange,D4_pchange,D5_pchange,D6_pchange,D7_pchange,D8_pchange,D9_pchange,D10_pchange,D11_pchange,D12_pchange,D13_pchange,D14_pchange,D15_pchange,D16_pchange,D17_pchange,D18_pchange,D19_pchange,D20_pchange],axis=1)
        
        result_df = pd.concat([result_df,result_series],axis=0)
        print('Selected codes are {}'.format(selected_codes))
        print('Getting return rate of {}'.format(return_rate))
    else:
        print('No stock is selected')
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time: ", execution_time, " seconds")
    print('\n')
    
end_time_all = time.time()
execution_time_all = end_time_all-start_time_original

print('Finish execution, using {} seconds'.format(execution_time_all))
if return_rate_total:
    average_return = sum(return_rate_total) / len(return_rate_total)
    print('Average return: {}'.format(average_return))
    portfolio_dict = {
        'Date':day_list,
        'Scode':code_list,
        'return_rate':return_rate_total
    }
    portfolio_df = pd.DataFrame(portfolio_dict)
    portfolio_df.to_excel('portfolio_return.xlsx')
    result_df['今日涨幅'] = result_df['今日涨幅'].astype(float)
    result_df['近三日涨幅'] = result_df['近三日涨幅'].astype(float)
    result_df['Ntimes'] = result_df['Ntimes'].astype(float)
    result_df['D+1涨幅'] = result_df['D+1涨幅'].astype(float)
    result_df['D+2涨幅'] = result_df['D+2涨幅'].astype(float)
    result_df['D+3涨幅'] = result_df['D+3涨幅'].astype(float)
    result_df['D+4涨幅'] = result_df['D+4涨幅'].astype(float)
    result_df['D+5涨幅'] = result_df['D+5涨幅'].astype(float)
    result_df['D+6涨幅'] = result_df['D+6涨幅'].astype(float)
    result_df['D+7涨幅'] = result_df['D+7涨幅'].astype(float)
    result_df['D+8涨幅'] = result_df['D+8涨幅'].astype(float)
    result_df['D+9涨幅'] = result_df['D+9涨幅'].astype(float)
    result_df['D+10涨幅'] = result_df['D+10涨幅'].astype(float)
    result_df['D+11涨幅'] = result_df['D+11涨幅'].astype(float)
    result_df['D+12涨幅'] = result_df['D+12涨幅'].astype(float)
    result_df['D+13涨幅'] = result_df['D+13涨幅'].astype(float)
    result_df['D+14涨幅'] = result_df['D+14涨幅'].astype(float)
    result_df['D+15涨幅'] = result_df['D+15涨幅'].astype(float)
    result_df['D+16涨幅'] = result_df['D+16涨幅'].astype(float)
    result_df['D+17涨幅'] = result_df['D+17涨幅'].astype(float)
    result_df['D+18涨幅'] = result_df['D+18涨幅'].astype(float)
    result_df['D+19涨幅'] = result_df['D+19涨幅'].astype(float)
    result_df['D+20涨幅'] = result_df['D+20涨幅'].astype(float)
    result_df['今日涨幅'] = result_df['今日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['近三日涨幅'] = result_df['近三日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['Ntimes'] = result_df['Ntimes'].map(lambda x: "{:.2f}".format(x))
    result_df['D+1涨幅'] = result_df['D+1涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+2涨幅'] = result_df['D+2涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+3涨幅'] = result_df['D+3涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+4涨幅'] = result_df['D+4涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+5涨幅'] = result_df['D+5涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+6涨幅'] = result_df['D+6涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+7涨幅'] = result_df['D+7涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+8涨幅'] = result_df['D+8涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+9涨幅'] = result_df['D+9涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+10涨幅'] = result_df['D+10涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+11涨幅'] = result_df['D+11涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+12涨幅'] = result_df['D+12涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+13涨幅'] = result_df['D+13涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+14涨幅'] = result_df['D+14涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+15涨幅'] = result_df['D+15涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+16涨幅'] = result_df['D+16涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+17涨幅'] = result_df['D+17涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+18涨幅'] = result_df['D+18涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+19涨幅'] = result_df['D+19涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+20涨幅'] = result_df['D+20涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['日期'] = pd.to_datetime(result_df['日期'], format='%Y-%m-%d')
    result_df.to_excel('result_df.xlsx')
else:
    print('No stock selected')

Selecting stocks at 2022-12-01 00:00:00
Selected codes are {'603057', '600763', '000587', '002411', '300015', '300957', '301267', '603288', '000799', '603866', '601137', '603755', '600887', '600105', '603369', '300896', '002812'}
Getting return rate of -0.0003124547512172171
Execution time:  0.6791863441467285  seconds


Selecting stocks at 2022-12-02 00:00:00
No stock is selected
Execution time:  0.13375496864318848  seconds


Selecting stocks at 2022-12-05 00:00:00
Selected codes are {'002222', '002324', '600367', '002050'}
Getting return rate of -0.0036850006824075132
Execution time:  0.7068467140197754  seconds


Selecting stocks at 2022-12-06 00:00:00
No stock is selected
Execution time:  0.13371729850769043  seconds


Selecting stocks at 2022-12-07 00:00:00
Selected codes are {'300938', '300269'}
Getting return rate of -0.040945250350959084
Execution time:  0.5696051120758057  seconds


Selecting stocks at 2022-12-08 00:00:00
No stock is selected
Execution time:  0.12357425689697

Selected codes are {'000617'}
Getting return rate of -0.02006688963210701
Execution time:  0.47458553314208984  seconds


Selecting stocks at 2023-02-24 00:00:00
No stock is selected
Execution time:  0.10427641868591309  seconds


Selecting stocks at 2023-02-27 00:00:00
Selected codes are {'002166'}
Getting return rate of 0.01615969581749055
Execution time:  0.484238862991333  seconds


Selecting stocks at 2023-02-28 00:00:00
Selected codes are {'600633', '600476', '300525', '002740', '300036', '300212', '000948', '000938'}
Getting return rate of 0.06478134524062673
Execution time:  0.48267340660095215  seconds


Selecting stocks at 2023-03-01 00:00:00
Selected codes are {'600122', '601311'}
Getting return rate of -0.015340364333652934
Execution time:  0.4904332160949707  seconds


Selecting stocks at 2023-03-02 00:00:00
Selected codes are {'603158'}
Getting return rate of -0.04722550177095641
Execution time:  0.48095250129699707  seconds


Selecting stocks at 2023-03-03 00:00:00
Selec

Selected codes are {'002876'}
Getting return rate of -0.005259006047857007
Execution time:  0.4929635524749756  seconds


Selecting stocks at 2023-05-08 00:00:00
Selected codes are {'603393'}
Getting return rate of -0.021333333333333315
Execution time:  0.4968116283416748  seconds


Selecting stocks at 2023-05-09 00:00:00
No stock is selected
Execution time:  0.10844993591308594  seconds


Selecting stocks at 2023-05-10 00:00:00
Selected codes are {'600226', '300680'}
Getting return rate of -0.014818140996856966
Execution time:  0.5088760852813721  seconds


Selecting stocks at 2023-05-11 00:00:00
No stock is selected
Execution time:  0.10329318046569824  seconds


Selecting stocks at 2023-05-12 00:00:00
Selected codes are {'002113'}
Getting return rate of -0.04761904761904767
Execution time:  0.5135021209716797  seconds


Selecting stocks at 2023-05-15 00:00:00
Selected codes are {'000918', '002150'}
Getting return rate of 0.057731958762886615
Execution time:  0.5988812446594238  seco

Selected codes are {'300863', '002649'}
Getting return rate of -0.06008174386921006
Execution time:  0.6366546154022217  seconds


Selecting stocks at 2023-07-20 00:00:00
Selected codes are {'300648'}
Getting return rate of -0.030937215650591487
Execution time:  0.6575634479522705  seconds


Selecting stocks at 2023-07-21 00:00:00
No stock is selected
Execution time:  0.14309310913085938  seconds


Selecting stocks at 2023-07-24 00:00:00
Selected codes are {'603322', '601966'}
Getting return rate of -0.005215375700212466
Execution time:  0.6349637508392334  seconds


Selecting stocks at 2023-07-25 00:00:00
Selected codes are {'601995', '002833', '000776', '600446', '000166', '600999', '600958', '002410', '001236', '600906', '601888'}
Getting return rate of -0.006161320987219887
Execution time:  0.6774086952209473  seconds


Selecting stocks at 2023-07-26 00:00:00
No stock is selected
Execution time:  0.14622783660888672  seconds


Selecting stocks at 2023-07-27 00:00:00
Selected codes 

Selected codes are {'301509', '301338', '000829', '300661', '002436', '603605', '300657', '603280', '603728'}
Getting return rate of 0.024265016176677534
Execution time:  0.6431324481964111  seconds


Selecting stocks at 2023-08-31 00:00:00
Selected codes are {'603337', '600744', '000821'}
Getting return rate of -0.005003573981415044
Execution time:  0.6538102626800537  seconds


Selecting stocks at 2023-09-01 00:00:00
Selected codes are {'002043', '300616', '002196', '301048', '300256'}
Getting return rate of -0.01269130272489749
Execution time:  0.6182253360748291  seconds


Selecting stocks at 2023-09-04 00:00:00
Selected codes are {'001322'}
Getting return rate of 0.005344418052256605
Execution time:  0.5888729095458984  seconds


Selecting stocks at 2023-09-05 00:00:00
No stock is selected
Execution time:  0.14017105102539062  seconds


Selecting stocks at 2023-09-06 00:00:00
No stock is selected
Execution time:  0.15061092376708984  seconds


Selecting stocks at 2023-09-07 00:00:

<ipython-input-23-27ec567b9df8>:197: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  portfolio_df.to_excel('portfolio_return.xlsx')
<ipython-input-23-27ec567b9df8>:245: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  result_df.to_excel('result_df.xlsx')


In [5]:
# X2=19.7
# set7 = set(today[(today['收盘']/today['滞后一期收盘']-1) < X2/100]['股票代码'])
# set7 = {code for code in set7 if not code.startswith('00')}
# set7
working_days

DatetimeIndex(['2023-11-10', '2023-11-13', '2023-11-14', '2023-11-15',
               '2023-11-16', '2023-11-17', '2023-11-20', '2023-11-21',
               '2023-11-22', '2023-11-23', '2023-11-24', '2023-11-27',
               '2023-11-28'],
              dtype='datetime64[ns]', freq='B')

In [8]:
date_string = '20231128'
timestamp = pd.to_datetime(date_string, format='%Y%m%d')
index = working_days.get_loc(timestamp)
return_df = new_total_df[new_total_df['股票代码']=='002043']
# get_filtered_df(,working_days[index+1], 10)
# D1_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df,working_days[index+1],1))).rename(columns={'收盘涨幅对数': 'D+1涨幅'}) # day+BDay(1)
# D1_pchange.astype(float)
# print(D1_pchange)
# get_filtered_df(return_df,working_days[index+1],1)

set_period = get_filtered_df(new_total_df, working_days[index], 10) #  day - BDay(prek)
period_vol_avg = get_average_vol(set_period)
vol_series = pd.concat([today_vol, period_vol_avg],axis=1).set_axis(['Series1', 'Series2'], axis=1)
volume_n_times_of_period_avg = vol_series[vol_series['Series1'] > Ntimes*vol_series['Series2']]
value_of_ntimes = vol_series['Series1']/vol_series['Series2']


In [10]:
value_of_ntimes

股票代码
000001    0.924760
000002    0.479954
000003         NaN
000004    0.614485
000005    0.470471
            ...   
873665    1.497557
873679    1.354050
873693    1.367154
873726    2.847509
873833    1.779230
Length: 5555, dtype: float64

In [39]:
# result_df['今日涨幅'] = result_df['今日涨幅'].astype(float)
# result_df['近三日涨幅'] = result_df['近三日涨幅'].astype(float)
# result_df['D+1涨幅'] = result_df['D+1涨幅'].astype(float)
# result_df['D+2涨幅'] = result_df['D+2涨幅'].astype(float)
# result_df['D+3涨幅'] = result_df['D+3涨幅'].astype(float)
# result_df['D+4涨幅'] = result_df['D+4涨幅'].astype(float)
# result_df['D+5涨幅'] = result_df['D+5涨幅'].astype(float)
# result_df['D+6涨幅'] = result_df['D+6涨幅'].astype(float)
# result_df['D+7涨幅'] = result_df['D+7涨幅'].astype(float)
# result_df['D+8涨幅'] = result_df['D+8涨幅'].astype(float)
# result_df['D+9涨幅'] = result_df['D+9涨幅'].astype(float)
# result_df['D+10涨幅'] = result_df['D+10涨幅'].astype(float)
# result_df['D+11涨幅'] = result_df['D+11涨幅'].astype(float)
# result_df['D+12涨幅'] = result_df['D+12涨幅'].astype(float)
# result_df['D+13涨幅'] = result_df['D+13涨幅'].astype(float)
# result_df['D+14涨幅'] = result_df['D+14涨幅'].astype(float)
# result_df['D+15涨幅'] = result_df['D+15涨幅'].astype(float)
# result_df['D+16涨幅'] = result_df['D+16涨幅'].astype(float)
# result_df['D+17涨幅'] = result_df['D+17涨幅'].astype(float)
# result_df['D+18涨幅'] = result_df['D+18涨幅'].astype(float)
# result_df['D+19涨幅'] = result_df['D+19涨幅'].astype(float)
# result_df['D+20涨幅'] = result_df['D+20涨幅'].astype(float)
# result_df['今日涨幅'] = result_df['今日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['近三日涨幅'] = result_df['近三日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+1涨幅'] = result_df['D+1涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+2涨幅'] = result_df['D+2涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+3涨幅'] = result_df['D+3涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+4涨幅'] = result_df['D+4涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+5涨幅'] = result_df['D+5涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+6涨幅'] = result_df['D+6涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+7涨幅'] = result_df['D+7涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+8涨幅'] = result_df['D+8涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+9涨幅'] = result_df['D+9涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+10涨幅'] = result_df['D+10涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+11涨幅'] = result_df['D+11涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+12涨幅'] = result_df['D+12涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+13涨幅'] = result_df['D+13涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+14涨幅'] = result_df['D+14涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+15涨幅'] = result_df['D+15涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+16涨幅'] = result_df['D+16涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+17涨幅'] = result_df['D+17涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+18涨幅'] = result_df['D+18涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+19涨幅'] = result_df['D+19涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+20涨幅'] = result_df['D+20涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['日期'] = pd.to_datetime(result_df['日期'], format='%Y-%m-%d')
# result_df.to_excel('result_df.xlsx')

<ipython-input-39-565c79222f58>:46: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  result_df.to_excel('result_df.xlsx')
